# Efficient Monte Carlo Simulation: Vectorized vs Non-Vectorized

This notebook benchmarks the performance difference between a loop-based Monte Carlo simulation and a NumPy vectorized implementation for pricing a European call option.

### Objectives
- Implement a non-vectorized Monte Carlo pricing function (Python loop)
- Implement a vectorized simulation using NumPy
- Benchmark execution time
- Compare pricing accuracy

Monte Carlo simulations can be computationally heavy when a large number of paths is required. Vectorization eliminates Python-level loops, significantly improving performance.

In [2]:
import sys
import os
sys.path.append(os.path.abspath(".."))

In [ ]:
import numpy as np
import time

In [4]:
S0 = 100
K = 105
r = 0.05
sigma = 0.2
T = 1.0
n_sim = 100_000
seed = 42

In [5]:
def mc_non_vectorized(S0, K, r, sigma, T, n_sim, seed=None):
    if seed is not None:
        np.random.seed(seed)
    payoffs = []
    for _ in range(n_sim):
        Z = np.random.normal()
        ST = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)
        payoffs.append(max(ST - K, 0))
    return np.exp(-r * T) * np.mean(payoffs)

In [6]:
def mc_vectorized(S0, K, r, sigma, T, n_sim, seed=None):
    if seed is not None:
        np.random.seed(seed)
    Z = np.random.normal(0, 1, n_sim)
    ST = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)
    payoffs = np.maximum(ST - K, 0)
    return np.exp(-r * T) * np.mean(payoffs)

In [7]:
start = time.time()
price_loop = mc_non_vectorized(S0, K, r, sigma, T, n_sim, seed)
t_loop = time.time() - start

start = time.time()
price_vec = mc_vectorized(S0, K, r, sigma, T, n_sim, seed)
t_vec = time.time() - start

print(f"Non-vectorized MC price: {price_loop:.4f}, time: {t_loop:.4f} sec")
print(f"Vectorized MC price:     {price_vec:.4f}, time: {t_vec:.4f} sec")
print(f"Speedup factor:          {t_loop / t_vec:.2f}x")

Non-vectorized MC price: 8.0416, time: 0.1524 sec
Vectorized MC price:     8.0416, time: 0.0038 sec
Speedup factor:          40.48x


In [8]:
difference = abs(price_loop - price_vec)
print(f"Absolute difference in price: {difference:.6f}")

Absolute difference in price: 0.000000


### Performance Summary

| Method           | Price    | Time (sec) | Speedup |
|------------------|----------|------------|---------|
| Loop-based       | ~8.0416  | ~0.1524    | 1x      |
| Vectorized       | ~8.0416  | ~0.0038    | >40x    |

The vectorized approach provides a substantial performance improvement with negligible loss in accuracy.

### Next Steps
- Implement time discretization for path-dependent options (e.g., Asian option)
- Integrate variance reduction techniques
- Consider Numba or multiprocessing to further optimize runtime
- Extend to other option types